In [232]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import re
import nltk

In [233]:
def read_text (file):
    with open(file,'r') as f:
        text = f.read()
    f.close()
    return text

In [234]:
bb = read_text('data/breitbart_pol_feb_20_26.txt')
nyt = read_text('data/nyt_pol_feb_20_26.txt')
hp = read_text('data/huffpost_pol_feb_20_26.txt')
fox = read_text('data/fox_pol_feb_20_26.txt')


In [235]:
d = [bb,nyt,fox,hp]
names = ["breitbart","nyt","fox","hp"]

In [236]:
stopWords = stopwords.words('english')
#stopWords = None
#stopWords.append('mr')
count_vect = CountVectorizer(stop_words = stopWords)

In [237]:
td_mat = count_vect.fit_transform(d)
print(td_mat.shape)
#count_vect.get_feature_names()

(4, 13435)


In [238]:
#tf and tf-idf
tf_transformer = TfidfTransformer(use_idf=False)
tf = tf_transformer.fit_transform(td_mat)
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(td_mat)

In [239]:
from sklearn.metrics.pairwise import linear_kernel

In [272]:
def cosine_similarities_mat(m):
    n = m.shape[0]
    cs_array = np.zeros((n,n))
    for i in range(n):
        cs_array[i]=(linear_kernel(m[i:i+1], m).flatten())
    return cs_array

In [249]:
cosine_similarities_mat(tfidf)

array([[ 1.        ,  0.59911043,  0.80491985,  0.8083596 ],
       [ 0.59911043,  1.        ,  0.63885702,  0.6384394 ],
       [ 0.80491985,  0.63885702,  1.        ,  0.84011819],
       [ 0.8083596 ,  0.6384394 ,  0.84011819,  1.        ]])

In [250]:
cosine_similarities_mat(tf)

array([[ 1.        ,  0.63301482,  0.84900263,  0.85859221],
       [ 0.63301482,  1.        ,  0.66559499,  0.67047691],
       [ 0.84900263,  0.66559499,  1.        ,  0.8800127 ],
       [ 0.85859221,  0.67047691,  0.8800127 ,  1.        ]])

In [244]:
names

['breitbart', 'nyt', 'fox', 'hp']

In [277]:
def pearson_corr_mat(m):
    n = m.shape[0]
    pc_array_all = np.corrcoef(m.toarray(), m.toarray())
    pc_array = np.zeros((n,n))
    for i in range(n):
        pc_array[i]=pc_array_all[i][0:4]
    return pc_array

In [278]:
pearson_corr_mat(tfidf)

array([[ 1.        ,  0.57399263,  0.7874664 ,  0.78691517],
       [ 0.57399263,  1.        ,  0.61783908,  0.6160966 ],
       [ 0.7874664 ,  0.61783908,  1.        ,  0.82579846],
       [ 0.78691517,  0.6160966 ,  0.82579846,  1.        ]])

In [279]:
pearson_corr_mat(tf)

array([[ 1.        ,  0.61496868,  0.83866086,  0.8466207 ],
       [ 0.61496868,  1.        ,  0.65020099,  0.65464212],
       [ 0.83866086,  0.65020099,  1.        ,  0.87194807],
       [ 0.8466207 ,  0.65464212,  0.87194807,  1.        ]])

In [217]:
stopwords = nltk.corpus.stopwords.words('english')
def tok(text):
    word_split_pattern = re.compile(r"\W|\d")
    words = re.split(word_split_pattern, text)
    words = list(filter(None,words))
    words=[w.lower() for w in words]
    words=[w for w in words if w not in stopwords]
    return words

In [218]:
tokenized_documents = [tok(x) for x in d]

In [220]:
def jaccard_similarity(d1, d2):
    intersection = set(d1).intersection(set(d2))
    union = set(d1).union(set(d2))
    return len(intersection)/len(union)